## Daten Laden

In [1]:
file_path = "LITTLE_2025-05-10__03_52_57-normalized-supplemented.csv"
print(file_path)

LITTLE_2025-05-10__03_52_57-normalized-supplemented.csv


In [2]:
import pandas as pd

df = pd.read_csv(file_path)

# Drop command column
df = df.drop(columns=['command'])

# Display the first few rows to understand the structure
df.head()

,watt-median,L1-dcache-loads:u,L1-dcache-loads:u-square,L1-dcache-stores:u,L1-dcache-stores:u-square,L1-icache-loads:u,L1-icache-loads:u-square,L1-icache-load-misses:u,L1-icache-load-misses:u-square,LLC-loads:u,...,ref-cycles:u,ref-cycles:u-square,topdown-bad-spec:u,topdown-bad-spec:u-square,topdown-be-bound:u,topdown-be-bound:u-square,topdown-fe-bound:u,topdown-fe-bound:u-square,topdown-retiring:u,topdown-retiring:u-square
0,0.00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000,...,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
1,8.45,1.508973e+09,2.276998e+18,6.066891e+08,3.680716e+17,1.436276e+09,2.062890e+18,2.813354e+06,7.914960e+12,975316.212134,...,3.096516e+09,9.588409e+18,8.361428e+08,6.991347e+17,1.238929e+09,1.534945e+18,2.035314e+09,4.142504e+18,7.357896e+09,5.413863e+19
2,7.26,6.174994e+08,3.813055e+17,2.788412e+08,7.775240e+16,8.123170e+08,6.598588e+17,5.543507e+05,3.073047e+11,400.366297,...,3.453882e+09,1.192930e+19,1.079781e+09,1.165927e+18,5.642234e+09,3.183481e+19,1.106342e+09,1.223993e+18,4.973468e+09,2.473539e+19
3,7.68,1.036649e+09,1.074641e+18,1.036638e+09,1.074618e+18,2.244483e+09,5.037702e+18,1.193041e+04,1.423346e+08,151.300356,...,3.462589e+09,1.198952e+19,4.591660e+08,2.108334e+17,1.538323e+09,2.366437e+18,3.805071e+08,1.447857e+17,1.050205e+10,1.102930e+20
4,7.00,1.830498e+08,3.350721e+16,1.830314e+08,3.350048e+16,1.832026e+08,3.356321e+16,9.597917e+03,9.212001e+07,189.327388,...,3.453620e+09,1.192749e+19,1.524338e+06,2.323606e+12,8.736575e+09,7.632775e+19,9.356647e+07,8.754684e+15,4.028857e+09,1.623169e+19


## Korrelationsmatrix

In [3]:
corr = df.corr()
corr.style.background_gradient(cmap='RdYlGn')

,watt-median,L1-dcache-loads:u,L1-dcache-loads:u-square,L1-dcache-stores:u,L1-dcache-stores:u-square,L1-icache-loads:u,L1-icache-loads:u-square,L1-icache-load-misses:u,L1-icache-load-misses:u-square,LLC-loads:u,LLC-loads:u-square,LLC-load-misses:u,LLC-load-misses:u-square,LLC-stores:u,LLC-stores:u-square,dTLB-loads:u,dTLB-loads:u-square,dTLB-load-misses:u,dTLB-load-misses:u-square,dTLB-stores:u,dTLB-stores:u-square,dTLB-store-misses:u,dTLB-store-misses:u-square,iTLB-load-misses:u,iTLB-load-misses:u-square,branch-loads:u,branch-loads:u-square,branch-load-misses:u,branch-load-misses:u-square,branch-instructions:u,branch-instructions:u-square,branch-misses:u,branch-misses:u-square,bus-cycles:u,bus-cycles:u-square,cache-misses:u,cache-misses:u-square,cache-references:u,cache-references:u-square,cpu-cycles:u,cpu-cycles:u-square,instructions:u,instructions:u-square,mem-loads:u,mem-loads:u-square,mem-stores:u,mem-stores:u-square,ref-cycles:u,ref-cycles:u-square,topdown-bad-spec:u,topdown-bad-spec:u-square,topdown-be-bound:u,topdown-be-bound:u-square,topdown-fe-bound:u,topdown-fe-bound:u-square,topdown-retiring:u,topdown-retiring:u-square
watt-median,1.000000,0.423071,0.246619,0.367177,0.260107,0.431859,0.281022,0.164221,0.160904,0.160537,0.160477,0.161686,0.160492,0.160546,0.160477,0.418663,0.236945,0.160867,0.160477,0.366216,0.259299,0.160695,0.160477,0.182869,0.161111,0.364272,0.228870,0.144219,0.074764,0.366502,0.230853,0.142499,0.073321,0.876255,0.848874,0.162439,0.160481,0.160661,0.160477,0.876028,0.848347,0.466999,0.344066,0.003264,0.002449,0.369547,0.264543,0.876825,0.850301,0.142078,0.078215,-0.102852,-0.165541,0.120349,0.032247,0.485436,0.352194
L1-dcache-loads:u,0.423071,1.000000,0.861010,0.798494,0.660537,0.347326,0.198760,0.154669,0.154961,0.154731,0.154840,0.153110,0.154834,0.154865,0.154840,0.999560,0.848455,0.154899,0.154840,0.803739,0.674592,0.154958,0.154840,0.178847,0.155827,0.383851,0.220423,0.090100,0.045443,0.404505,0.235194,0.088872,0.044619,0.083955,0.068274,0.155141,0.154841,0.154869,0.154840,0.083815,0.068024,0.434897,0.366181,-0.033435,-0.028095,0.789683,0.643174,0.084450,0.069141,0.075358,0.026189,-0.341488,-0.392954,-0.000829,-0.100992,0.361622,0.286681
L1-dcache-loads:u-square,0.246619,0.861010,1.000000,0.664311,0.667411,0.194656,0.103142,0.073481,0.077978,0.078858,0.078930,0.077174,0.078920,0.078852,0.078930,0.871168,0.999433,0.078942,0.078930,0.676539,0.693928,0.078934,0.078930,0.083884,0.079052,0.203090,0.102401,-0.010114,-0.012608,0.234264,0.124187,-0.010528,-0.012739,0.035926,0.027762,0.078922,0.078929,0.078892,0.078930,0.035858,0.027643,0.337582,0.316191,-0.060180,-0.043875,0.644446,0.629431,0.037166,0.030149,-0.033925,-0.027676,-0.235958,-0.265577,-0.060663,-0.078509,0.246944,0.198658
L1-dcache-stores:u,0.367177,0.798494,0.664311,1.000000,0.924191,0.409208,0.314691,0.066180,0.067890,0.068312,0.068406,0.067968,0.068412,0.068361,0.068406,0.798157,0.653558,0.068412,0.068406,0.999573,0.923238,0.068473,0.068406,0.081839,0.068844,0.476692,0.353879,0.056846,0.094427,0.491001,0.364022,0.057452,0.095392,0.074096,0.065931,0.068581,0.068407,0.068389,0.068406,0.074050,0.065846,0.480835,0.449041,-0.066736,-0.032364,0.999256,0.926590,0.076297,0.070245,0.000333,0.060727,-0.429696,-0.415065,0.126758,0.004335,0.436201,0.400882
L1-dcache-stores:u-square,0.260107,0.660537,0.667411,0.924191,1.000000,0.334531,0.282185,-0.000656,0.003867,0.005062,0.005120,0.004180,0.005118,0.005000,0.005120,0.664309,0.661790,0.005098,0.005120,0.925024,0.998850,0.005116,0.005120,0.004443,0.004889,0.374419,0.284229,-0.023922,0.023142,0.391684,0.296125,-0.023358,0.023838,0.040962,0.039262,0.005053,0.005119,0.005054,0.005120,0.040972,0.039277,0.420513,0.419142,-0.106611,-0.071270,0.919396,0.997876,0.043244,0.043782,-0.081561,-0.005191,-0.378248,-0.338212,0.130386,0.051405,0.373806,0.358791
L1-icache-loads:u,0.431859,0.347326,0.194656,0.409208,0.334531,1.000000,0.953655,0.136564,0.138455,0.138535,0.138562,0.137433,0.138562,0.138558,0.138562,

## 1. Versuch (Regression)

In [4]:
import pandas as pd
from sklearn.feature_selection import SelectKBest, f_regression
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error

# Zielvariable: Annahme, dass die erste Spalte die zu schätzende Zahl enthält
y = df.iloc[:, 0]

# Feature, das immer genutzt werden soll: 'instructions'
X_instructions = df[['instructions:u']]

# Potenzielle zusätzliche Features: Alle Spalten außer der Zielvariable und 'instructions'
potential_features = df.drop(columns=[df.columns[0], 'instructions:u'])

# Automatische Auswahl der 4 besten Features basierend auf f_regression
selector = SelectKBest(score_func=f_regression, k=4)
X_selected = selector.fit_transform(potential_features, y)

# Ausgabe der Namen der ausgewählten Features
selected_feature_names = potential_features.columns[selector.get_support()]
print("Ausgewählte Features:", selected_feature_names.tolist())

# Falls gewünscht: Umwandeln in DataFrame, um die Spaltennamen beizubehalten
X_selected_df = pd.DataFrame(X_selected, columns=selected_feature_names, index=df.index)

# Kombination der 'instructions'-Spalte mit den ausgewählten Features
X_combined = pd.concat([X_instructions, X_selected_df], axis=1)

# Aufteilen in Trainings- und Testdaten
X_train, X_test, y_train, y_test = train_test_split(X_combined, y, test_size=0.2, random_state=42)

# Training eines Regressionsmodells (z. B. Lineare Regression)
model = LinearRegression()
model.fit(X_train, y_train)

# Vorhersagen und Evaluation
y_pred = model.predict(X_test)
print("Mean Squared Error:", mean_squared_error(y_test, y_pred))


Ausgewählte Features: ['bus-cycles:u', 'cpu-cycles:u', 'ref-cycles:u', 'ref-cycles:u-square']
Mean Squared Error: 1375.8233698087524


## 2. Versuch (Regression)

### Modell trainieren

In [5]:
import pandas as pd
from sklearn.linear_model import LinearRegression
from sklearn.feature_selection import SequentialFeatureSelector
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error

# Zielvariable: erste Spalte (angenommen, es handelt sich um numerische Werte)
y = df.iloc[:, 0]

# Fixes Feature: 'instructions'
X_instructions = df[['instructions:u']]

# Kandidaten für die automatische Feature-Auswahl:
# Entfernen der Zielvariable und der Spalte 'instructions'
X_candidates = df.drop(columns=[df.columns[0], 'instructions:u'])

# Basisregressor für die Feature-Selektion (hier: Lineare Regression)
model = LinearRegression()

# Sequential Feature Selector (forward selection) wählt 4 zusätzliche Features aus
sfs = SequentialFeatureSelector(
    model,
    n_features_to_select=4,
    direction='forward',
    scoring='neg_mean_squared_error',
    cv=5  # 5-fache Kreuzvalidierung
)

# Durchführung der Feature-Selektion auf den Kandidatenfeatures
sfs.fit(X_candidates, y)

# Ermitteln der Namen der ausgewählten Features
selected_feature_names = X_candidates.columns[sfs.get_support()]
print("Ausgewählte Features:", selected_feature_names.tolist())

# Zusammenführen des fixen Features 'instructions' mit den ausgewählten Features
X_selected = pd.concat([X_instructions, X_candidates[selected_feature_names]], axis=1)

# Aufteilen in Trainings- und Testdaten
X_train, X_test, y_train, y_test = train_test_split(X_selected, y, test_size=0.2, random_state=42)

# Modelltraining auf den kombinierten Features
model.fit(X_train, y_train)

# Vorhersagen und Evaluation
y_pred = model.predict(X_test)
print("Mean Squared Error:", mean_squared_error(y_test, y_pred))

# Ausgabe der Regressionsparameter
print("\nRegression Parameter:")
print("Achsenabschnitt (Intercept):", model.intercept_)

# Ausgabe der Koeffizienten mit den zugehörigen Feature-Namen
print("Koeffizienten:")
for feature, coef in zip(X_selected.columns, model.coef_):
    print(f"{feature}: {coef}")

Ausgewählte Features: ['L1-dcache-stores:u-square', 'cpu-cycles:u', 'mem-stores:u', 'topdown-retiring:u']
Mean Squared Error: 13.172522035260432

Regression Parameter:
Achsenabschnitt (Intercept): 14.03972404470409
Koeffizienten:
instructions:u: 1.0402657931032004e-11
L1-dcache-stores:u-square: -2.865194270406296e-19
cpu-cycles:u: -2.8657251131317597e-09
mem-stores:u: 9.22763097581691e-10
topdown-retiring:u: 5.797137704672484e-11


### Modell testen

In [6]:
def do_regression(instructions, L1_dcache_loads, L1_icache_loads, topdown_retiring, uncore_clock_clockticks):
  return 5.671202142645853 + instructions * -4.037434477408138e-11 + L1_dcache_loads * 3.7727498541646285e-10 + topdown_retiring * 9.090283761268329e-11 + uncore_clock_clockticks * 1.4513036090171807e-09

# sleep 10 (expected 2.14)
print("sleep 10 (expected 2.14)")
print(do_regression(165556.153198144, 44181.0547038958, 54556.2364921857, 208105.052597547, 38000929.1425307))

print("")

# sum_up_benchmark (expected 10.46)
print("sum_up_benchmark (expected 10.46)")
print(do_regression(7658080659.63333, 1628055678.50196, 1528910300.65665, 7961570136.8392, 2696560318.9496))

print("")

# stress-ng --cpu 2 --cpu-method all --timeout 10s (expected 11.055)
print("stress-ng --cpu 2 --cpu-method all --timeout 10s (expected 11.055)")
print(do_regression(9439119304.88239, 1242411937.11209, 1632001751.87898, 9934301707.13731, 2690003038.87471))

sleep 10 (expected 2.14)
5.726381929781779

sum_up_benchmark (expected 10.46)
10.613493876265267

stress-ng --cpu 2 --cpu-method all --timeout 10s (expected 11.055)
10.565902164387403


## 3. Versuch (Schwierig auf Colab...)

### Modell trainieren

In [7]:
import pandas as pd
import numpy as np
from itertools import combinations
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import GridSearchCV, train_test_split
from sklearn.metrics import mean_squared_error
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.pipeline import Pipeline

class CustomSelector(BaseEstimator, TransformerMixin):
    """
    Transformer, der die Spalte 'instructions' immer mit einbezieht
    und zusätzlich eine Liste von weiteren Features auswählt, die als Parameter übergeben wird.
    """
    def __init__(self, features):
        self.features = features

    def fit(self, X, y=None):
        return self

    def transform(self, X):
        # Es werden immer 'instructions' plus die angegebenen features zurückgegeben
        return X[['instructions:u'] + list(self.features)]

def optimal_feature_selection_regression_cv(df, test_size=0.2, random_state=42):
    """
    Verwendet GridSearchCV, um die optimale Feature-Kombination zu finden.
    'instructions' wird stets genutzt und zusätzlich werden 4 weitere Features
    aus den Kandidaten (alle außer der Zielvariable und 'instructions') ausgewählt.

    Rückgabe:
      dict: Mit den besten zusätzlichen Features, dem CV-Score,
            Test-MSE, Achsenabschnitt und Koeffizienten.
    """
    # Zielvariable: erste Spalte
    y = df.iloc[:, 0]
    # Prädiktoren: alle Spalten außer der Zielvariable
    X = df.drop(columns=[df.columns[0]])

    # Kandidaten für zusätzliche Features: alle außer 'instructions'
    candidates = list(X.drop(columns=['instructions:u']).columns)

    # Pipeline: CustomSelector + LinearRegression
    pipeline = Pipeline([
        ('selector', CustomSelector(features=[])),
        ('regressor', LinearRegression())
    ])

    # Erzeuge den Parametergrid: alle Kombinationen von 4 Features aus den Kandidaten
    param_grid = {
        'selector__features': [list(comb) for comb in combinations(candidates, 4)]
    }

    grid = GridSearchCV(pipeline, param_grid, cv=5, scoring='neg_mean_squared_error', n_jobs=-1)
    grid.fit(X, y)

    best_features = grid.best_params_['selector__features']
    best_score = grid.best_score_

    print("Beste zusätzliche Features:", best_features)
    print("Bester CV Score (neg_mean_squared_error):", best_score)

    # Aufteilen in Trainings- und Testdaten
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=test_size, random_state=random_state)

    # Bestes Modell trainieren
    best_model = grid.best_estimator_
    best_model.fit(X_train, y_train)

    y_pred = best_model.predict(X_test)
    mse = mean_squared_error(y_test, y_pred)
    print("Finales Test-MSE:", mse)

    # Regressionsparameter ausgeben
    regressor = best_model.named_steps['regressor']
    print("\nRegression Parameter:")
    print("Achsenabschnitt (Intercept):", regressor.intercept_)

    coef_dict = {}
    # Die verwendeten Spalten: 'instructions:u' plus die best_features
    features_used = ['instructions:u'] + best_features
    print("Koeffizienten:")
    for feature, coef in zip(features_used, regressor.coef_):
        print(f"{feature}: {coef}")
        coef_dict[feature] = coef

    return {
        "best_features": best_features,
        "cv_score": best_score,
        "test_mse": mse,
        "intercept": regressor.intercept_,
        "coefficients": coef_dict
    }

# Beispiel zur Nutzung:
# df = pd.read_csv('data.csv')
# result = optimal_feature_selection_regression_cv(df)
# print(result)

optimal_feature_selection_regression_cv(df)

Beste zusätzliche Features: ['L1-dcache-loads:u', 'L1-icache-loads:u', 'iTLB-load-misses:u', 'bus-cycles:u']
Bester CV Score (neg_mean_squared_error): -0.05778050498261268
Finales Test-MSE: 0.681758101424615

Regression Parameter:
Achsenabschnitt (Intercept): 3.1605309472304457
Koeffizienten:
instructions:u: 3.6757760305158816e-11
L1-dcache-loads:u: 3.2089460350838357e-10
L1-icache-loads:u: 1.7825941074649793e-10
iTLB-load-misses:u: 2.910827313533828e-06
bus-cycles:u: 1.0259849984396058e-09


{'best_features': ['L1-dcache-loads:u',
  'L1-icache-loads:u',
  'iTLB-load-misses:u',
  'bus-cycles:u'],
 'cv_score': np.float64(-0.05778050498261268),
 'test_mse': 0.681758101424615,
 'intercept': np.float64(3.1605309472304457),
 'coefficients': {'instructions:u': np.float64(3.6757760305158816e-11),
  'L1-dcache-loads:u': np.float64(3.2089460350838357e-10),
  'L1-icache-loads:u': np.float64(1.7825941074649793e-10),
  'iTLB-load-misses:u': np.float64(2.910827313533828e-06),
  'bus-cycles:u': np.float64(1.0259849984396058e-09)}}

### Modell testen

In [8]:
def do_regression(instructions, L1_dcache_stores, dTLB_loads, dTLB_stores, uncore_clock_clockticks):
  return 20.835376283133087 + instructions * 7.719055169945736e-11 + L1_dcache_stores * 6.750946704125803e-09 + dTLB_loads * 5.752769284893428e-10 + dTLB_stores * -6.910438337198133e-09 + uncore_clock_clockticks * -4.169191670737849e-09

# sleep 10 (expected 2.14)
print("sleep 10 (expected 2.14)")
print(do_regression(165556.153198144, 26701.7192578545, 44249.7939094154, 26902.9815858347, 38000929.1425307))

print("")

# sum_up_benchmark (expected 10.46)
print("sum_up_benchmark (expected 10.46)")
print(do_regression(7658080659.63333, 698890004.553369, 1626382945.67966, 698381494.410173, 2696560318.9496))

print("")

# stress-ng --cpu 2 --cpu-method all --timeout 10s (expected 11.055)
print("stress-ng --cpu 2 --cpu-method all --timeout 10s (expected 11.055)")
print(do_regression(9439119304.88239, 561931144.341175, 1237133407.39931, 559806745.508526, 2690003038.87471))

sleep 10 (expected 2.14)
20.67697571161636

sum_up_benchmark (expected 10.46)
11.011698438164549

stress-ng --cpu 2 --cpu-method all --timeout 10s (expected 11.055)
10.985600363889704
